# pandas 进阶修炼 ｜早起Python
<br>

**pandas进阶修炼系列由公众号【早起Python & 可视化图鉴】 原创**

**转载及其他形式合作请与我们联系（微信号`sshs321`)，未经授权严禁搬运及二次创作，侵权必究！**


本习题基于 `pandas` 版本 `1.1.3`，所有内容应当在 `Jupyter Notebook` 中执行以获得最佳效果。

不同版本之间写法可能会有少许不同，如若碰到此情况，你应该学会如何自行检索解决。

## 项目简介



<br>


**<font color = '#5172F0'><font size=3.5>必读👇👇👇</font>**
    
**本习题微信公众号「<font color ='#F77802'>可以叫我才哥</font>」友情提供，授权公众号「早起Python」二次整理发布**


    
### 关于 RFM 模型

根据美国数据库营销研究所 `Arthur Hughes` 的研究，客户数据库中有3个神奇的要素，这3个要素构成了数据分析最好的指标。

- R：最近一次消费间隔 (`Recency`)，计算用户最近一次消费记录截止当前时间的间隔天数
- F：消费频率 (`Frequency`)，计算在统计周期内用户消费记录次数
- M：消费金额 (`Monetary`)，计算在统计周期内用户累计消费金额或者是单次平均消费金额

针对RFM，我们可以发现：

- 如果最近一次消费间隔R越小，就表示用户上次消费至今最近，那么该用户的流失风险越低
- 如果消费频率F越大，就表示用户在统计周期内消费次数多，那么该用户的忠诚度越高
- 如果消费金额M越大，就表示用户在统计周期内消费支出的金额多，那么该用户的价值越高
    
基于以上RFM的值，我们简单的按照大小高低来划分，可以分为8类用户群体。
![](./picture/1.png)

为了方便划分，我们可以简单根据 `RFM` 各自的均值来判断高低：

- 高：大于均值
- 低：不大于均值
 
从而我们可以很好地区分出8类群体，这8类用户群体的特征如下表所示，我们可以根据自己产品的现状制定出更适合不同用户群体的业务决策，从而提升数据。
![](./picture/2.png)


### 项目流程

本文将基于游戏数据，进行 `RFM` 分析，`RFM` 建模流程：

1. 数据预处理
2. 指标打分
3. 计算RFM
4. RFM客户分群
    
**<font color = '#5172F0'><font size=3.5>更多资源👇👇👇</font>**   
    
如果需要获得与本案例更多的资源，可以微信搜索公众号「<font color ='#F77802'>可以叫我才哥</font>」关注，或者点击下方文章链接查看！

- [使用Python简单玩玩RFM用户价值模型](https://mp.weixin.qq.com/s/jTPJ9THYzZ8DJhOs0IqkiA)

### 特别说明

<br>

该 `Notebook` 版本为**习题+答案版**

因本文项目较为复杂，且部分习题很难正确表述，因此答案将直接附在习题下方。

你应该思考是否有更优解。

## 项目实战

### 1 - 加载数据

导入当前文件夹内 `数据.csv`

In [30]:
import pandas as pd

df = pd.read_csv('数据.csv')

### 2 - 数据预览

查看数据前 5 行

In [15]:
df.head()

,@timestamp,price,uid
0,2021/4/11 23:59,48,94244483
1,2021/4/11 23:58,8,94228493
2,2021/4/11 23:58,200,94244423
3,2021/4/11 23:57,48,94244423
4,2021/4/11 23:56,48,94243723


### 3 - 类型查看

查看数据基本信息

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   @timestamp  100000 non-null  object
 1   price       100000 non-null  int64 
 2   uid         100000 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 2.3+ MB


原始数据中，`@timestamp`是时间、`price`是价格、`uid`是用户唯一标识符。

### 4 - 类型转换

我们发现时间字段类似并非时间，这里需要进行转化，转化的时候需要注意只获取日期即可，时间部分不需要。

In [17]:
df['@timestamp'] = pd.to_datetime(df['@timestamp']).dt.floor('d')
df.head()

,@timestamp,price,uid
0,2021-04-11,48,94244483
1,2021-04-11,8,94228493
2,2021-04-11,200,94244423
3,2021-04-11,48,94244423
4,2021-04-11,48,94243723


### 5 - 描述性信息

查看数据描述行信息

In [18]:
df.describe()

,price,uid
count,100000.000000,1.000000e+05
mean,187.057440,6.741167e+07
std,484.823679,2.628848e+07
min,8.000000,1.000023e+06
25%,8.000000,5.762843e+07
50%,48.000000,7.879366e+07
75%,240.000000,8.526513e+07
max,5184.000000,9.424448e+07


### 6 - 计算 R值 F值和M值

通过R、F、M的定义计算其值，直接分组聚合计算即可得到F和M的值。

In [19]:
data = df.groupby('uid').agg(
                            last_date=('@timestamp','max'), # 计算最近一次消费日期
                            F=('@timestamp','count'), # 计算消费次数 F
                            M=('price','sum'), # 计算消费总金额（也可选择平均值） M
                            ).reset_index()
data.head()

,uid,last_date,F,M
0,1000023,2021-04-02,1,240
1,1000063,2021-04-06,1,8
2,1000583,2021-03-29,1,8
3,1000873,2021-03-28,1,8
4,1000923,2021-03-12,2,96


### 7 - 计算最近一次消费间隔R

直接利用当前日期减去最近消费日期可得到最近消费间隔

In [22]:
data['R'] = (pd.to_datetime('2021.4.12') - data['last_date']).dt.days 
data.head()

,uid,last_date,F,M,R
0,1000023,2021-04-02,1,240,10
1,1000063,2021-04-06,1,8,6
2,1000583,2021-03-29,1,8,14
3,1000873,2021-03-28,1,8,15
4,1000923,2021-03-12,2,96,31


### 8 - 数据分组打分

根据上面计算出来的RFM值，我们按照一定的分箱规则进行赋分即可，关于这个打分的规则，大家根据自己产品类型、历史数据或行业经验自行判断即可。

我这边简单做了如下划分：

R打分标准：

| 分值 | R区间 | 说明                 |
| :--- | :---- | :------------------- |
| 1    | 0-7   | 最近7天内有充值行为  |
| 2    | 7~14  |                      |
| 3    | 14~21 |                      |
| 4    | 21~28 |                      |
| 5    | 28~   | 超过28天没有充值行为 |

F打分标准

| 分值 | F区间 | 说明              |
| :--- | :---- | :---------------- |
| 1    | 1     | 周期内充值1次     |
| 2    | 2     |                   |
| 3    | 3     |                   |
| 4    | 4     |                   |
| 5    | 4+    | 周期内充值4次以上 |

M打分标准：

| 分值 | M区间   | 说明                |
| :--- | :------ | :------------------ |
| 1    | 100-    | 周期内充值100元以内 |
| 2    | 100~200 |                     |
| 3    | 200~400 |                     |
| 4    | 400~800 |                     |
| 5    | 800+    | 周期内充值800元以上 |


这里，我们直接采用分箱进行操作即可

需要注意的是由于分箱的返回结果类型是`Categoricals`类型，无法用于后续进行分值算术运算，需要进行类型转化。

In [23]:
# 打分，直接参考统计区间
# R-score （7天以内，5分；7-14天，4分；14-21天，3分；21-28天，2分；超过28天，1分）
data['R_score'] = pd.cut(data['R'],
                         bins=[0,7,14,21,28,10000],
                         labels=[5,4,3,2,1]
                        ).astype('int')

data['F_score'] = pd.cut(data['F'],
                         bins=[0,1,2,3,4,10000],
                         labels=[1,2,3,4,5]
                        ).astype('int')

data['M_score'] = pd.cut(data['M'],
                         bins=[0,100,200,400,800,1000000],
                         labels=[1,2,3,4,5]
                        ).astype('int')

data.head()

,uid,last_date,F,M,R,R_score,F_score,M_score
0,1000023,2021-04-02,1,240,10,4,1,3
1,1000063,2021-04-06,1,8,6,5,1,1
2,1000583,2021-03-29,1,8,14,4,1,1
3,1000873,2021-03-28,1,8,15,3,1,1
4,1000923,2021-03-12,2,96,31,1,2,1


### 9 - 数据划分

我们完成对指标打分之后，按照8类用户群体划分的方式。

直接比较各种的平均值即可获得高低分类，用1表示高、0表示低

In [24]:
data['R_level'] = (data['R_score']>data['R_score'].mean())*1
data['F_level'] = (data['F_score']>data['F_score'].mean())*1
data['M_level'] = (data['M_score']>data['M_score'].mean())*1

data.head()

,uid,last_date,F,M,R,R_score,F_score,M_score,R_level,F_level,M_level
0,1000023,2021-04-02,1,240,10,4,1,3,1,0,1
1,1000063,2021-04-06,1,8,6,5,1,1,1,0,0
2,1000583,2021-03-29,1,8,14,4,1,1,1,0,0
3,1000873,2021-03-28,1,8,15,3,1,1,1,0,0
4,1000923,2021-03-12,2,96,31,1,2,1,0,1,0


### 10 - 计算 RFM

In [25]:
data['RFM'] = data['R_level'].astype('str').str.cat([data['F_level'].astype('str'),data['M_level'].astype('str')])

data.head()

,uid,last_date,F,M,R,R_score,F_score,M_score,R_level,F_level,M_level,RFM
0,1000023,2021-04-02,1,240,10,4,1,3,1,0,1,101
1,1000063,2021-04-06,1,8,6,5,1,1,1,0,0,100
2,1000583,2021-03-29,1,8,14,4,1,1,1,0,0,100
3,1000873,2021-03-28,1,8,15,3,1,1,1,0,0,100
4,1000923,2021-03-12,2,96,31,1,2,1,0,1,0,010


### 11 - RFM用户分群

基于8类用户分群规则，这里直接用replace函数方法进行操作

In [26]:
data['RFM'] = data['RFM'].replace(['111','101','011','001','110','100','010','000'],
                                  ['重要价值用户','重要发展用户','重要保持用户','重要挽留用户','一般价值用户','一般发展用户','一般保持用户','一般挽留用户'])

data.head()

,uid,last_date,F,M,R,R_score,F_score,M_score,R_level,F_level,M_level,RFM
0,1000023,2021-04-02,1,240,10,4,1,3,1,0,1,重要发展用户
1,1000063,2021-04-06,1,8,6,5,1,1,1,0,0,一般发展用户
2,1000583,2021-03-29,1,8,14,4,1,1,1,0,0,一般发展用户
3,1000873,2021-03-28,1,8,15,3,1,1,1,0,0,一般发展用户
4,1000923,2021-03-12,2,96,31,1,2,1,0,1,0,一般保持用户


### 12 - 数据计算

计算8类用户群体数量分布：

In [27]:
data.groupby('RFM')['uid'].nunique().to_frame('用户数').reset_index()

,RFM,用户数
0,一般价值用户,3214
1,一般保持用户,2896
2,一般发展用户,13332
3,一般挽留用户,14137
4,重要价值用户,6022
5,重要保持用户,5069
6,重要发展用户,2950
7,重要挽留用户,3857


### 13 - 数据可视化

In [29]:
import plotly.express as px

dataRFM = data.groupby('RFM')['uid'].nunique().to_frame('用户数').reset_index()
fig = px.bar(dataRFM, x='RFM', y='用户数',
             color='用户数', # 指定柱状图颜色根据 用户数字段数值大小自动着色
             height=600, # 图表高度
             text = '用户数',
             title= '最近42天付费用户RFM分群人数分布',
            )
fig.update_traces(
    textposition='outside',
) 
fig.show()

以上就是本次游戏用户 RFM 分析的全部内容～

![](http://liuzaoqi.oss-cn-beijing.aliyuncs.com/2021/09/16/16317972442543.jpg?域名/sample.jpg?x-oss-process=style/stylename)

![](./picture/3.png)